In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d



In [ ]:
trials=100_00_00     #total simulated events
low=-1;high=1;interval=high-low
test=interval*np.random.rand(trials,3)+low
#test
x=test[:,0]
y=test[:,1]
z=test[:,2]

In [ ]:
x

In [ ]:
y

In [ ]:
z

In [ ]:

r=np.sqrt(x**2+y**2+z**2)
r

In [ ]:
test=np.column_stack((test,r))

In [ ]:
df=pd.DataFrame(test,columns=['x','y','z','r']) #data frame with x,y,z and r
df.head()

In [ ]:
df_less=df[df.r<0.5]          #data frame with r<0.5 (radius of the bead)
df_less.head()
total_counts=df_less.shape[0]
total_counts

In [ ]:
%matplotlib inline
df_sample=df_less.head(1000)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(df_sample.x,df_sample.y,df_sample.z)

In [ ]:
#this function give the normal distance from a point to the surface of the sphere

def get_upward_distance(x,y,z,dz=0.001,r=0.5):
    '''
    returns normal distance from a point in the sphere to the surface as well as the z co-ordinate in the sphere
    '''
    #z=np.abs(z)
    d=np.sqrt(x**2+y**2+z**2)
    if z==-0.5: return 1.0,0.5 #z_up, z
    z_up=0
    #print(f"before while z_up: {z_up}")
    while(d<r):
        z+=dz
        #print(f"Inside while z: {z}")
        z_up+=dz
        d=np.sqrt(x**2+y**2+z**2)
    #print(f"after while z_up: {z_up}")
    return z_up,z
        
    

In [ ]:
%%time
#distance travelled in the bead
df_less_copy=df_less.copy()

df_less_copy.loc[:,'z_new']=df_less_copy.apply(lambda x:get_upward_distance(x['x'],x['y'],x['z'])[1],axis=1)
df_less_copy.loc[:,'d_bead']=df_less_copy.apply(lambda x:get_upward_distance(x['x'],x['y'],x['z'])[0],axis=1)
df_less_copy.head()

In [ ]:
#distance travelled in the button
button_thickness=3.0
df_less_copy.loc[:,'d_button']=button_thickness/2.-df_less_copy.loc[:,'z_new'] #Thickness of button is 3 mm and thickness above 0 is 1.5 mm
df_less_copy.head()

In [ ]:
#changing mm to cm
mm_to_cm=0.1
df_less_copy=df_less_copy*mm_to_cm
df_less_copy.head()

$ P_{transmission} = e^{-\frac{x}{\lambda}} $ <br>

x is thickness and $\lambda$ is attenuation length
<br> $\lambda = \frac{1}{\rho \cdot \mu} $ <br>
$ P_{transmission} = e^{-\mu \cdot \rho \cdot x} $ <br>

$ \mu $ ($cm^2/g$) <br> $ \rho $ ($g/cm^3$) <br> $\lambda (cm)$ <br>
$ \mu$ from https://physics.nist.gov/PhysRefData/Xcom/html/xcom1.html
<br>
Tranmission factor from: https://www.ezag.com/home/products/isotope_products/isotrak_calibration_sources/reference_sources/gamma_sources/point_sources/#c601

# fMaterialsManager->AddMaterial("Acrylic","C5-H8-O2",1.18*g/cm3,"")
# ceramic: O7410-Na300-Ca60-Mg10-Al600-Si1600-Pb4


![image](beadcomposition.png)

In [ ]:
#Probabality of scatering from the button

button_density = 1.10         #.18        #g/cm^3 (from simulation data)
#button_mu_pb   = 0.1976       #C5H8O2 for 47keV (0.047 MeV) gamma rays
bead_mu_pb     = 3.195E-01    #47 keV pb-21O for ceramic=> O7410Na300Ca60Mg10Al600Si1600Pb4
bead_mu_am     = 2.314E-01    #60 keV am-241 for ceramic=> O7410Na300Ca60Mg10Al600Si1600Pb4

button_mu_pb =    1.797E-01     #C870O98N32
button_mu_am =    1.669E-01     #60 keV Am-241



#trnasmission factors from manufacturer
transmission_pb=0.964   #47 keV pb-210

transmission_am=0.958   #60 keV am-241

#Am-241 0.958


#button_mu_am=0.1815    #60 keV am-241
#lambda calculation for acrylic button  (C5H802)
#fMaterialsManager->AddMaterial("Acrylic","C5-H8-O2",1.18*g/cm3,"");
#density is from oldmc




def transmission_prob(x,mu,rho):
    '''reutrns button transmission probability'''
    #return (1-np.exp(-1.0*x*mu*rho))
    return (np.exp(-1.0*x*mu*rho))

    
    
    

In [ ]:
#adding the transmississin though button
df_less_copy.loc[:,'button_transmission_prob_pb']=df_less_copy.apply(lambda x:transmission_prob(x['d_button'],button_mu_pb,button_density),axis=1)
df_less_copy.loc[:,'button_transmission_prob_am']=df_less_copy.apply(lambda x:transmission_prob(x['d_button'],button_mu_am,button_density),axis=1)
df_less_copy.head()

In [ ]:
#creating the random number bewteen 0 and 1, if the prob_product is greater than random number add 1 else add 0
def random_assign(x):
    if x>np.random.rand():
        return 1
    return 0
    

In [ ]:
#adding the transmississin though button
bead_density=2.55#g/cm^3  
df_less_copy.loc[:,'bead_transmission_prob_pb']=df_less_copy.apply(lambda x:transmission_prob(x['d_bead'],bead_mu_pb,bead_density),axis=1)
df_less_copy.loc[:,'bead_transmission_prob_am']=df_less_copy.apply(lambda x:transmission_prob(x['d_bead'],bead_mu_am,bead_density),axis=1)
df_less_copy.head()

In [ ]:
N_generated=df_less.shape[0]
#df_less['N_emitted']=df_less['button_transmission_prob']*df_less['bead_transmission_prob']*N_generated
df_less_copy['prob_product_pb']=df_less_copy['button_transmission_prob_pb']*df_less_copy['bead_transmission_prob_pb']
df_less_copy['prob_product_am']=df_less_copy['button_transmission_prob_am']*df_less_copy['bead_transmission_prob_am']
df_less_copy.head()

In [ ]:
#avg_of_prob_pb=round(df_less['prob_product_pb'].mean(),3)
avg_of_prob_am=round(df_less_copy['prob_product_am'].mean(),3)
avg_of_prob_am

In [ ]:
avg_random=round(df_less_copy['prob_product_am'].apply(lambda x:random_assign(x)).mean(),3)
avg_random

In [ ]:
def get_prob_am(bead_density):
    '''
    returns the net prob of transmission for the chosen density of bead for am-241
    '''
    df_less_copy['bead_transmission_prob_am']=df_less_copy.apply(lambda x:transmission_prob(x['d_bead'],bead_mu_am,bead_density),axis=1)
    df_less_copy['prob_product_am']=df_less_copy['button_transmission_prob_am']*df_less_copy['bead_transmission_prob_am']
    #avg_prob_am=round(df_less_copy['prob_product_am'].mean(),7)
    avg_prob_am=df_less_copy['prob_product_am'].mean()
    return [bead_density,avg_prob_am]

def get_prob_pb(bead_density):
    '''
    returns the net prob of transmission for the chosen density of bead for pb-210
    '''
    df_less_copy['bead_transmission_prob_pb']=df_less_copy.apply(lambda x:transmission_prob(x['d_bead'],bead_mu_pb,bead_density),axis=1)
    df_less_copy['prob_product_pb']=df_less_copy['button_transmission_prob_pb']*df_less_copy['bead_transmission_prob_pb']
    #avg_prob_am=round(df_less_copy['prob_product_pb'].mean(),7)
    avg_prob_am=df_less_copy['prob_product_pb'].mean()
    return [bead_density,avg_prob_am]

In [ ]:
print(f"Prob of am {transmission_am}\n")
density_prob_am=[]
for i in np.linspace(0,5,50):
    test=get_prob_am(i)
    density_prob_am.append(test)
    print(f"test :{test}")
#df_less.head()
    

In [ ]:
print(f"Prob of pb {transmission_pb}\n")
density_prob_pb=[]
for i in np.linspace(0,5,50):
    test=get_prob_pb(i)
    density_prob_pb.append(test)
    print(f"test :{test}")
#df_less.head()
    

In [ ]:
density_prob_am

In [ ]:
density_prob_pb

In [ ]:
#adding the columns to the single
transmission_pb=0.964   #47 keV pb-210

transmission_am=0.958   #60 keV am-241
all_densities=np.column_stack((density_prob_am,density_prob_pb))
columns=["density","am-prob","pb-prob"]
df_den=pd.DataFrame(all_densities[:,[0,1,3]],columns=columns)
df_den["am(0.958)-diff"]=df_den["am-prob"]-transmission_am
df_den["pb(0.964)-diff"]=df_den["pb-prob"]-transmission_pb
df_den["diff-sum"]=df_den["am(0.958)-diff"].abs()+df_den["pb(0.964)-diff"].abs()
df_den["diff-sum-quad"]=df_den["am(0.958)-diff"]**2+df_den["pb(0.964)-diff"]**2
#df_den=df_den.round(12)
df_den.to_csv("combined_densities.csv",index=False)
df_den

In [ ]:
df_den[(df_den['am-prob'].round(3)==transmission_am) | (df_den['pb-prob'].round(3)==transmission_pb)]

In [ ]:
minn,maxx=df_den['diff-sum-quad'].min(),df_den['diff-sum-quad'].max()

In [ ]:
df_den[(df_den['diff-sum-quad']==minn)]

In [ ]:
df_den.iloc[16:18,:]

In [ ]:
df_den.iloc[16:18,:].mean()

# Densities in the GEANT SIMULATIONS <br> pb-210 = 1.22449 $g/cm^3$ <br> am-241 = 2.55102 $g/cm^3$ <br> min-quad-error density = 1.683673 $g/cm^3$

In [ ]:
df_den.loc[df_den['am(0.958)-diff']==df_den['am(0.958)-diff'].min()]

In [ ]:
df_den.loc[df_den['diff-sum-quad']==maxx]

In [ ]:
#read the file
df=pd.read_csv('combined_densities.csv')
df

In [ ]:
df_abs=df.copy().abs()
cols=df_abs.columns.to_list()
x,y1,y2=df_abs[cols[0]].to_list(),df_abs[cols[3]].to_list(),df_abs[cols[4]].to_list()
x[:5],y1[:5],y2[:5]

In [ ]:
%matplotlib qtline
am_transmission=0.958
pb_transmission=0.964
fig,ax=plt.subplots(2,1,figsize=(5,4))
ax[0].plot(x,y1,'r.',label='am-diff')
ax[0].plot(x,y2,'b.',label='pb-diff')
ax[0].set_xlabel('density')
ax[0].set_ylabel('density difference')

ax[0].axvline(x=am_transmission,color='green',linestyle='--')
ax[0].axvline(x=pb_transmission,color='violet',linestyle='--')
ax[0].set_xlim(0,3)
ax[0].grid(which='both')
ax[0].legend()

ax[1].plot(x,y1,'r.',label='am-diff')
ax[1].plot(x,y2,'b.',label='pb-diff')
ax[1].set_xlabel('density')
ax[1].set_ylabel('density difference')

ax[1].axvline(x=am_transmission,color='green',linestyle='--')
ax[1].axvline(x=pb_transmission,color='violet',linestyle='--')
ax[1].set_xlim(0,1.05)
ax[1].grid(which='both')
ax[1].lege

nd()
plt.tight_layout()


In [ ]:
df_abs=df.copy().abs()
cols=df_abs.columns.to_list()
x,y1,y2=df_abs[cols[0]].to_list(),df_abs[cols[3]].to_list(),df_abs[cols[4]].to_list()
x[:5],y1[:5],y2[:5]

In [ ]:
%matplotlib qtline
am_transmission=0.958
pb_transmission=0.964
fig,ax=plt.subplots(2,1,figsize=(5,4))
ax[0].plot(x,y1,'r.',label='am-diff')
ax[0].plot(x,y2,'b.',label='pb-diff')
ax[0].set_xlabel('density')
ax[0].set_ylabel('density difference')

ax[0].axvline(x=am_transmission,color='green',linestyle='--')
ax[0].axvline(x=pb_transmission,color='violet',linestyle='--')
ax[0].set_xlim(0,3)
ax[0].grid(which='both')
ax[0].legend()

ax[1].plot(x,y1,'r.',label='am-diff')
ax[1].plot(x,y2,'b.',label='pb-diff')
ax[1].set_xlabel('density')
ax[1].set_ylabel('density difference')

ax[1].axvline(x=am_transmission,color='green',linestyle='--')
ax[1].axvline(x=pb_transmission,color='violet',linestyle='--')
ax[1].set_xlim(0,1.05)
ax[1].grid(which='both')
ax[1].lege

nd()
plt.tight_layout()


In [ ]:
df_abs